In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline

In [2]:
class Generator(object):
    flip=False
    
    def __init__(self, path_x, path_y, batch_size=16):
        self.path_x = path_x
        self.path_y = path_y
        self.batch_size = batch_size
        
        self.x_files = os.listdir(self.path_x)
        self.x_files.sort()
        
        self.y = np.loadtxt(os.path.join(self.path_y,"y"))
        
        self.pop_list = np.arange(self.y.shape[0])
          
    def __next__(self):
        selected = np.random.choice(self.pop_list, replace=False, size = self.batch_size)
        
        x = []
        for c,select in enumerate(selected):
            tmp = np.load(os.path.join(self.path_x, str(select).zfill(5)+".npy"))
            x.append(tmp)
        x = np.array(x)
        
        if self.flip:
            x = np.rollaxis(x,3,1)
            
        x = x.astype(np.float32)
        x /= 255
            
        return x, self.y[selected]
        

batch_size = 16
num_steps = 10000000

path_x = "cifar/cifar_npy"
path_y = "cifar/cifar_labels/"
generator = Generator(path_x = path_x,
                      path_y = path_y,
                      batch_size = batch_size)

generator.flip=True

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class G(nn.Module):
    def __init__(self):
        super(G, self).__init__()
        self.dr_rate = .2
        
        self.convT_1 = nn.ConvTranspose2d(in_channels=256,
                                            out_channels=256,
                                            kernel_size=(8,8),
                                            padding=0,
                                            output_padding=0,
                                            stride=1,
                                            bias=False)
        self.bn_1 = nn.BatchNorm2d(256)
        
        self.convT_2 = nn.ConvTranspose2d(in_channels=256,
                                            out_channels=128,
                                            kernel_size=5,
                                            padding=2,
                                            output_padding=1,
                                            stride=2,
                                            bias=True)
        
        self.convT_3 = nn.ConvTranspose2d(in_channels=128,
                                            out_channels=64,
                                            kernel_size=5,
                                            padding=2,
                                            output_padding=1,
                                            stride=2,
                                            bias=False)
        self.bn_3 = nn.BatchNorm2d(64)
        
        self.conv1 = nn.Conv2d(in_channels=64,
                                out_channels=3,
                                kernel_size=3,
                                padding=1,
                                stride=1,
                                bias=True)
        
    def forward(self, z):
        dropout = nn.Dropout(p=self.dr_rate)
        z = dropout(F.leaky_relu(self.convT_1(z), negative_slope=0.1))
        z = self.bn_1(z)
        
        z = dropout(F.leaky_relu(self.convT_2(z), negative_slope=0.1))
        
        z = F.leaky_relu(self.convT_3(z), negative_slope=0.1)
        z = self.bn_3(z)
        
        z = F.sigmoid(self.conv1(z))   
        return z
        
g = G()
g.cuda()

G(
  (convT_1): ConvTranspose2d (256, 256, kernel_size=(8, 8), stride=(1, 1), bias=False)
  (bn_1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (convT_2): ConvTranspose2d (256, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
  (convT_3): ConvTranspose2d (128, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1), bias=False)
  (bn_3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (conv1): Conv2d (64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [4]:
class D(nn.Module):
    def __init__(self):
        super(D, self).__init__()
        self.dr_rate = 0.2
        
        self.conv1 = nn.Conv2d(in_channels=3,
                               out_channels=32, 
                               kernel_size=3, 
                               stride=2,
                               padding=1)
        
        self.conv2 = nn.Conv2d(in_channels=32,
                               out_channels=64, 
                               kernel_size=3, 
                               stride=2,
                               padding=1)
        
        self.conv3 = nn.Conv2d(in_channels=64,
                               out_channels=128, 
                               kernel_size=3, 
                               stride=2,
                               padding=1)
        
        self.fc1 = nn.Linear(in_features=128 * 4 * 4, 
                             out_features=512)
        
        self.fc2 = nn.Linear(in_features=512,
                             out_features=1)

    
    def dr_rate(self, dr_rate):
        self.dr_rate = dr_rate

    def forward(self, x):
        dropout = nn.Dropout(p=self.dr_rate)
        # Convolutional Part
        x = dropout(F.leaky_relu(self.conv1(x), negative_slope=.1))
        x = dropout(F.leaky_relu(self.conv2(x), negative_slope=.1))
        x = dropout(F.leaky_relu(self.conv3(x), negative_slope=.1))
        
        # Flatten
        x = x.view(-1, 128 * 4 * 4)
        # Dense Layers
        x = dropout(F.relu(self.fc1(x)))
        x = F.sigmoid(self.fc2(x))
        return x

d = D()
d.cuda()

D(
  (conv1): Conv2d (3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d (32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d (64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (fc1): Linear(in_features=2048, out_features=512)
  (fc2): Linear(in_features=512, out_features=1)
)

In [5]:
latent_size = 256
criterion = nn.BCELoss()
d_optimizer = optim.Adam(d.parameters(), lr=0.0001)
g_optimizer = optim.Adam(g.parameters(), lr=0.0001)

In [41]:
from tqdm import tqdm
for c in tqdm(range(num_steps)):
    d.zero_grad()

    # Train d on real
    x_train, _ = next(generator)
    x_train = Variable(torch.from_numpy(x_train).cuda())
    y_train = torch.ones(batch_size, 1)
    y_train = Variable(y_train.cuda())

    y_pred = d(x_train)
    d_loss = criterion(y_pred, y_train)
    d_loss.backward()


     # Train d on fake
    z = torch.Tensor(batch_size, latent_size, 1, 1).normal_(0, 1)
    z = Variable(z.cuda(), requires_grad=False)
    y_train = torch.zeros(batch_size, 1)
    y_train = Variable(y_train.cuda())

    x_train = g(z).detach()
    y_pred = d(x_train)
    d_loss = criterion(y_pred, y_train)
    d_loss.backward()

    d_optimizer.step()

    # Train g
    g.zero_grad()

    z = torch.Tensor(batch_size, latent_size, 1, 1).normal_(0, 1)
    z = Variable(z.cuda(), requires_grad=False)
    y_train = torch.ones(batch_size, 1)
    y_train = Variable(y_train.cuda())

    x_train = g(z)
    y_pred = d(x_train)
    g_loss = criterion(y_pred, y_train)
    g_loss.backward()  

    g_optimizer.step()
    
    if c%10000 ==0:
        z = torch.Tensor(16, latent_size, 1, 1).normal_(0, 1)
        z = Variable(z.cuda(), requires_grad=False)
        y_train = torch.zeros(batch_size, 1)
        y_train = Variable(y_train.cuda())

        tmp = g(z).detach()
        x = np.array(tmp.data)
        x = np.rollaxis(x,1,4)


        plt.figure(figsize = (4,4))
        gs1 = gridspec.GridSpec(4, 4)
        gs1.update(wspace=0.0, hspace=0.0) # set the spacing between axes. 

        for i in range(16):
           # i = i + 1 # grid spec indexes from 0
            ax1 = plt.subplot(gs1[i])

            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
            ax1.set_xticks([])
            ax1.set_yticks([])
            ax1.set_aspect('equal')
            ax1.imshow(x[i])

        plt.savefig("progress_dog/{}.png".format(c), bbox_inches='tight', dpi=200)
        plt.close()
    
    
    


  0%|          | 11/10000000 [00:00<198:54:12, 13.97it/s]
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/mnt/fs2/home/alook/anaconda2/envs/torch/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/mnt/fs2/home/alook/anaconda2/envs/torch/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/mnt/fs2/home/alook/anaconda2/envs/torch/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  3%|▎         | 336106/10000000 [1:59:26<57:14:08, 46.90it/s]


KeyboardInterrupt: 

In [ ]:
torch.save(g.state_dict(),"models/callback_g_dog")
torch.save(d.state_dict(),"models/callback_d_dog")